In [ ]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from vae.datasets import VolSurfaceDataSetDict, VolSurfaceExFeatsDataSet
from vae.cvae_with_mem import CVAEMem
from vae.utils import *

In [ ]:
set_seeds(0)
torch.set_default_dtype(torch.float64)

In [ ]:
seq_len = 5
ctx_len = seq_len - 1
num_epochs = 1

In [ ]:
data = np.load("data/vol_surface_with_ret.npz")
vol_surf_data = data["surface"]
ret_data = data["ret"]
skew_data = data["skews"]
slope_data = data["slopes"]
ex_data = np.concatenate([ret_data[...,np.newaxis], skew_data[...,np.newaxis], slope_data[...,np.newaxis]], axis=-1)
train_simple = DataLoader(VolSurfaceDataSetDict(vol_surf_data[:4000], seq_len), shuffle=True, batch_size=64)
valid_simple = DataLoader(VolSurfaceDataSetDict(vol_surf_data[4000:5000], seq_len), shuffle=True, batch_size=16)
test_simple = DataLoader(VolSurfaceDataSetDict(vol_surf_data[5000:], seq_len), shuffle=True, batch_size=16)

train_ex = DataLoader(VolSurfaceExFeatsDataSet(vol_surf_data[:4000], ex_data[:4000], seq_len), shuffle=True, batch_size=64)
valid_ex = DataLoader(VolSurfaceExFeatsDataSet(vol_surf_data[4000:5000], ex_data[4000:5000], seq_len), shuffle=True, batch_size=16)
test_ex = DataLoader(VolSurfaceExFeatsDataSet(vol_surf_data[5000:], ex_data[5000:], seq_len), shuffle=True, batch_size=16)

# Conv version

## with ex feat

In [ ]:
config = {
    "seq_len": seq_len, 
    "feat_dim": (5, 5),
    "latent_dim": 5,
    "device": "cuda",
    "kl_weight": 1,
    "re_feat_weight": 0,
    "surface_hidden": [2, 4, 5],
    "ex_feats_dim": 3,
    "ex_feats_hidden": None,
    "mem_type": "lstm",
    "mem_hidden": 75,
    "mem_layers": 2,
    "mem_dropout": 0.2,
    "ctx_len": ctx_len, 
    "ctx_surface_hidden": [2, 4, 5], 
    "ctx_ex_feats_hidden": None,
    "interaction_layers": None,
    "use_dense_surface": False,
}
model = CVAEMem(config)
print(model)

In [ ]:
train(model, train_ex, valid_ex, epochs=num_epochs, lr=1e-05, model_dir="test_spx/ex", file_name="conv2d_mem_spx_ex.pt")

In [ ]:
test(model, valid_ex, test_ex, "test_spx/ex/conv2d_mem_spx_ex.pt")

In [ ]:
surf, ex_feat = model.get_surface_given_conditions({"surface": torch.from_numpy(vol_surf_data[5000:5000+ctx_len]), 
                                    "ex_feats": torch.from_numpy(ex_data[5000:5000+ctx_len])})
surf = surf.detach().cpu().numpy().reshape((5,5))
ex_feat = ex_feat.detach().cpu().numpy()
print(f"Real ex: {ex_data[5000+ctx_len]}, predicted ex: {ex_feat}")
plot_surface_separate(vol_surf_data[5000+ctx_len], surf)

In [ ]:
model_data = torch.load("test_spx/ret/conv2d_mem_spx_ret.pt")
sim = generate_surface_spx(vol_surf_data, ret_data, model_data, start_time=5000, steps_to_sim=30, model_type=CVAEMem)
plot_surface_time_series(sim)

## no ex feats

In [ ]:
config = {
    "seq_len": seq_len, 
    "feat_dim": (5, 5),
    "latent_dim": 5,
    "device": "cuda",
    "kl_weight": 1,
    "re_feat_weight": 0,
    "surface_hidden": [2, 4, 5],
    "ex_feats_dim": 0,
    "ex_feats_hidden": None,
    "mem_type": "lstm",
    "mem_hidden": 75,
    "mem_layers": 2,
    "mem_dropout": 0.2,
    "ctx_len": ctx_len, 
    "ctx_surface_hidden": [2, 4, 5], 
    "ctx_ex_feats_hidden": None,
    "interaction_layers": None,
    "use_dense_surface": False,
}
model = CVAEMem(config)
print(model)

In [ ]:
train(model, train_simple, valid_simple, epochs=num_epochs, lr=1e-05, model_dir="test_spx/simple", file_name="conv2d_mem_spx.pt")

In [ ]:
test(model, valid_simple, test_simple, "test_spx/simple/conv2d_mem_spx.pt")

In [ ]:
surf = model.get_surface_given_conditions({"surface": torch.from_numpy(vol_surf_data[5000:5000+ctx_len])})
surf = surf.detach().cpu().numpy().reshape((5,5))
plot_surface_separate(vol_surf_data[5000+ctx_len], surf)

In [ ]:
model_data = torch.load("test_spx/simple/conv2d_mem_spx.pt")
sim = generate_surface_spx(vol_surf_data, None, model_data, start_time=5000, steps_to_sim=30, model_type=CVAEMem)
plot_surface_time_series(sim)

# Dense version

## ret as ex feat

In [ ]:
config = {
    "seq_len": seq_len, 
    "feat_dim": (5, 5),
    "latent_dim": 25,
    "device": "cuda",
    "kl_weight": 1,
    "re_feat_weight": 0,
    "surface_hidden": [25, 50, 50],
    "ex_feats_dim": 1,
    "ex_feats_hidden": None,
    "mem_type": "lstm",
    "mem_hidden": 75,
    "mem_layers": 2,
    "mem_dropout": 0.2,
    "ctx_len": ctx_len, 
    "ctx_surface_hidden": [25, 50, 50], 
    "ctx_ex_feats_hidden": None,
    "interaction_layers": None,
    "use_dense_surface": True,
}
model = CVAEMem(config)
print(model)

In [ ]:
train(model, train_ex, valid_ex, epochs=num_epochs, lr=1e-05, model_dir="test_spx/ret", file_name="dense_mem_spx_ret.pt")

In [ ]:
test(model, valid_ex, test_ex, "test_spx/ret/dense_mem_spx_ret.pt")

In [ ]:
surf, ex_feat = model.get_surface_given_conditions({"surface": torch.from_numpy(vol_surf_data[5000:5000+ctx_len]), 
                                    "ex_feats": torch.from_numpy(ret_data[5000:5000+ctx_len]).unsqueeze(-1)})
surf = surf.detach().cpu().numpy().reshape((5,5))
ex_feat = ex_feat.detach().cpu().numpy().reshape((1,))[0]
print(f"Real return: {ret_data[5000+ctx_len]}, predicted: {ex_feat}")
plot_surface_separate(vol_surf_data[5000+ctx_len], surf)

In [ ]:
model_data = torch.load("test_spx/ret/dense_mem_spx_ret.pt")
sim = generate_surface_spx(vol_surf_data, ret_data, model_data, start_time=5000, steps_to_sim=30, model_type=CVAEMem)
plot_surface_time_series(sim)

## no ex feats

In [ ]:
config = {
    "seq_len": seq_len, 
    "feat_dim": (5, 5),
    "latent_dim": 25,
    "device": "cuda",
    "kl_weight": 1,
    "re_feat_weight": 0,
    "surface_hidden": [25, 50, 50],
    "ex_feats_dim": 0,
    "ex_feats_hidden": None,
    "mem_type": "lstm",
    "mem_hidden": 75,
    "mem_layers": 2,
    "mem_dropout": 0.2,
    "ctx_len": ctx_len, 
    "ctx_surface_hidden": [25, 50, 50], 
    "ctx_ex_feats_hidden": None,
    "interaction_layers": None,
    "use_dense_surface": True,
}
model = CVAEMem(config)
print(model)

In [ ]:
train(model, train_simple, valid_simple, epochs=num_epochs, lr=1e-05, model_dir="test_spx/simple", file_name="dense_mem_spx.pt")

In [ ]:
test(model, valid_simple, test_simple, "test_spx/simple/dense_mem_spx.pt")

In [ ]:
surf = model.get_surface_given_conditions({"surface": torch.from_numpy(vol_surf_data[5000:5000+ctx_len])})
surf = surf.detach().cpu().numpy().reshape((5,5))
plot_surface_separate(vol_surf_data[5000+ctx_len], surf)

In [ ]:
model_data = torch.load("test_spx/simple/dense_mem_spx.pt")
sim = generate_surface_spx(vol_surf_data, None, model_data, start_time=5000, steps_to_sim=30, model_type=CVAEMem)
plot_surface_time_series(sim)